# Boltzmann Machines

### Importing the libraries

In [13]:
import numpy as np
import pandas as pd
import torch
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable


### Importing the dataset

In [14]:
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1',
                     names=['movie_id', 'title', 'category'])
users = pd.read_csv('ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1',
                    names=['user_id', 'gender', 'age', 'user_job_id', 'zip_code'])
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1',
                      names=['user_id', 'movie_id', 'rating', 'timestamp'])


### Preparing the training set and the test set

In [15]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t',
                           names=['user_id', 'movie_id', 'rating', 'timestamp'])
training_set = np.array(training_set, dtype='int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter='\t',
                       names=['user_id', 'movie_id', 'rating', 'timestamp'])
test_set = np.array(test_set, dtype='int')


### Getting the number of users and movies

In [16]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))


In [17]:
movies

,movie_id,title,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


### Converting the data into an array with users in lines and movies in columns

In [18]:

def convert(data):
    new_data = []
    for user_id in range(1, nb_users + 1):
        id_movies = data[:, 1][data[:, 0] == user_id]
        id_ratings = data[:, 2][data[:, 0] == user_id]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data


### Array with users in lines and movies in columns
### [user_1, user_2, ..., user_943]
### [[movie_1_rating] [movie_2_rating] ... [movie_1682_rating]]

In [19]:
training_set = convert(training_set)
test_set = convert(test_set)


### Converting the data into Torch tensors

In [20]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)


### Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)

In [21]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1



### Creating the architecture of the Neural Network

In [22]:


class RBM:
    def __init__(self, n_visible_nodes, n_hidden_nodes):
        self.W = torch.randn(n_hidden_nodes, n_visible_nodes)
        self.a = torch.randn(1, n_hidden_nodes)
        self.b = torch.randn(1, n_visible_nodes)

    def sample_h(self, x):
        """
        Sample the probabilities of the hidden nodes given
        the visible nodes
        :param x: inputs
        :return:
        """
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)  # Weights + bias
        p_h_given_v = torch.sigmoid(activation)  # output
        return p_h_given_v, torch.bernoulli(p_h_given_v)

    def sample_v(self, y):
        """
        Sample the probabilities of the visibles nodes given the
        hidden nodes
        :param y:
        :return:
        """
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)  # Weights + bias
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)

    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)



In [23]:
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)


### Training the RBM

In [24]:
nb_epoch = 40
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user + batch_size]
        v0 = training_set[id_user:id_user + batch_size]
        ph0, _ = rbm.sample_h(v0)
        for k in range(10):
            _, hk = rbm.sample_h(vk)
            _, vk = rbm.sample_v(hk)
            vk[v0 < 0] = v0[v0 < 0]
        phk, _ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
        s += 1.
    print('epoch: ' + str(epoch) + ' loss: ' + str(train_loss / s))


epoch: 1 loss: tensor(0.3680)
epoch: 2 loss: tensor(0.2522)
epoch: 3 loss: tensor(0.2396)
epoch: 4 loss: tensor(0.2487)
epoch: 5 loss: tensor(0.2510)
epoch: 6 loss: tensor(0.2463)
epoch: 7 loss: tensor(0.2489)
epoch: 8 loss: tensor(0.2500)
epoch: 9 loss: tensor(0.2485)
epoch: 10 loss: tensor(0.2507)
epoch: 11 loss: tensor(0.2465)
epoch: 12 loss: tensor(0.2486)
epoch: 13 loss: tensor(0.2509)
epoch: 14 loss: tensor(0.2483)
epoch: 15 loss: tensor(0.2492)
epoch: 16 loss: tensor(0.2488)
epoch: 17 loss: tensor(0.2471)
epoch: 18 loss: tensor(0.2474)
epoch: 19 loss: tensor(0.2479)
epoch: 20 loss: tensor(0.2476)
epoch: 21 loss: tensor(0.2489)
epoch: 22 loss: tensor(0.2470)
epoch: 23 loss: tensor(0.2469)
epoch: 24 loss: tensor(0.2500)
epoch: 25 loss: tensor(0.2472)
epoch: 26 loss: tensor(0.2488)
epoch: 27 loss: tensor(0.2469)
epoch: 28 loss: tensor(0.2465)
epoch: 29 loss: tensor(0.2475)
epoch: 30 loss: tensor(0.2484)
epoch: 31 loss: tensor(0.2459)
epoch: 32 loss: tensor(0.2465)
epoch: 33 loss: t

### Testing the RBM

In [25]:

test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user + 1]
    vt = test_set[id_user:id_user + 1]
    if len(vt[vt >= 0]) > 0:
        _, h = rbm.sample_h(v)
        _, v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt >= 0] - v[vt >= 0]))
        s += 1.
print('test loss: ' + str(test_loss / s))


test loss: tensor(0.2454)
